# PeptideQ to Protein
### Approach: Merge by protein accession \#s
* Note that a possible problem would be that a given sequence can be associated with multiple proteins. This means that in the PeptideQ file, protein accession numbers are list under "Protein Groups"
* To remedy this we may need to split the Protein Groups into individual columns per protein accession \#... This may not be very useful though because there is an unknown maximum \# of proteins per group
* depending on how important this is we can add a catch for that but for the moment I'll leave it like this


In [51]:
import pandas as pd
import numpy as np
import os
import pyteomics.mzml
import spectrum_utils.spectrum as sus
from pathlib import Path

In [97]:
def save_file(psm_dataframe, file_path):
    psm_dataframe.to_csv(file_path, sep="\t", index=False)
    print(f'Dataframe saved.')


def load_peptideQ(peptideQ_file_path):
    peptideQ_dataframe = pd.read_table(peptideQ_file_path, delimiter='\t')
    #print(peptideQ_dataframe.columns)
    # rename the "Protein Groups" header so we can use the df.merge function later
    peptideQ_dataframe = peptideQ_dataframe.rename({"Protein Groups": "Protein Accession", "Sequence" : "Peptide"}, axis=1)
    #print(peptideQ_dataframe.columns)
    return peptideQ_dataframe

def load_protein(protein_file_path):
    # load the protein file into a pandas dataframe
    protein_dataframe = pd.read_table(protein_file_path)
    return protein_dataframe
    
def load_msfragger_peptideQ(peptideQ_file_path):
    peptideQ_dataframe = pd.read_table(peptideQ_file_path, delimiter='\t')
    # rename the "Protein Groups" header so we can use the df.merge function later
    peptideQ_dataframe = peptideQ_dataframe.rename({'Protein ID': 'Protein Accession', 'Peptide Sequence': 'Peptide'}, axis=1)
    return peptideQ_dataframe

def load_msfragger_protein(protein_file_path):
    # load the protein file into a pandas dataframe
    protein_dataframe = pd.read_table(protein_file_path)

    # Rename the "Protein ID" column to faciliate merging
    protein_dataframe = protein_dataframe.rename({'Protein ID': 'Protein Accession'}, axis=1)
    
    return protein_dataframe
    


In [103]:
def join_peptideQ_and_protein_dataframes(protein_df, peptideQ_df):
    # join based on the "Protein Accession"
    joined_dataframe = peptideQ_df.merge(right=protein_df, on="Protein Accession", how='inner', suffixes=('_protein', '_peptide'))
    
    # generate multiIndex
    joined_dataframe = joined_dataframe.set_index(['Protein Accession', 'Peptide'])
    
    return joined_dataframe

In [126]:
def peptideQ_and_protein_controller (file_type, peptideQ_file_path, protein_file_path, peptideQ_and_protein_file_path, columns_to_keep=None):
    
    ''' Joins a Peptide and Protein Quantification files. Files are joined into a pandas dataframe and saved as a tsv.
    
    Required Parameters:
        * file_type: "mm" for metamorpheus files, or "msfragger" for msfragger files
        * peptideQ_file_path: File path to the Peptide Quantification file
        * protein_file_path: File path to the Protein Quantification file
        * peptideQ_and_protein_file_path: Output file path
        
    Optional Parameters: 
        * columns_to_keep: List of columns to include in the dataframe. Note that column names may vary based on whether your files were generated with MetaMorpheus or MSFragger. '''

    # load dataframes
    if file_type.lower() == 'mm':
        protein_df = load_protein(protein_file_path)
        peptideQ_df = load_peptideQ(peptideQ_file_path)
    elif file_type.lower() == 'msfragger':
        peptideQ_df = load_msfragger_peptideQ(peptideQ_file_path=peptideQ_file_path)
        protein_df = load_msfragger_protein(protein_file_path=protein_file_path)
    else:
        print('invalid file type')
        return
    

    # create joined dataframe and save as csv
    joined_df = join_peptideQ_and_protein_dataframes(protein_df=protein_df, peptideQ_df=peptideQ_df)

    # select all columns to keep, if this parameter was not passed in, return dataframe with all columns
    if columns_to_keep != None:
        joined_df = joined_df[columns_to_keep]

    save_file(joined_df, file_path=peptideQ_and_protein_file_path)

    return joined_df

In [127]:
help(controller)

Help on function controller in module __main__:

controller(file_type, peptideQ_file_path, protein_file_path, peptideQ_and_protein_file_path, columns_to_keep=None)
    Joins a Peptide and Protein Quantification files. Files are joined into a pandas dataframe and saved as a tsv.
    
    Required Parameters:
        * file_type: "mm" for metamorpheus files, or "msfragger" for msfragger files
        * peptideQ_file_path: File path to the Peptide Quantification file
        * protein_file_path: File path to the Protein Quantification file
        * peptideQ_and_protein_file_path: Output file path
        
    Optional Parameters: 
        * columns_to_keep: List of columns to include in the dataframe. Note that column names may vary based on whether your files were generated with MetaMorpheus or MSFragger.



In [105]:
# testing the parser
protein_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\2ng\\AllQuantifiedProteinGroups.tsv"
peptideQ_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\2ng\\AllQuantifiedPeptides.tsv"
outfile_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\02ng\\peptideQ_and_protein.tsv"
test_df = controller(file_type='mm',peptideQ_file_path=peptideQ_file_path, protein_file_path=protein_file_path, peptideQ_and_protein_file_path=outfile_path)


Dataframe saved.


In [106]:
test_df

Base Sequence  \
Protein Accession Peptide                                                        
Q99453            AAAAAAAAAK                                        AAAAAAAAAK   
P37108            AAAAAAAAAPAAAATAPTTAATTAATAAQ  AAAAAAAAAPAAAATAPTTAATTAATAAQ   
                  FQMAYSNLLR                                        FQMAYSNLLR   
                  GTVEGFEPADNK                                    GTVEGFEPADNK   
                  ISTVVSSK                                            ISTVVSSK   
...                                                                        ...   
Q14865            YPESLSR                                              YPESLSR   
Q9BQP7            YSNLVQSVLSSR                                    YSNLVQSVLSSR   
O95167            YSVMINK                                              YSVMINK   
Q9UBM7            YTAAVPYR                                            YTAAVPYR   
Q8NAV1            YVLEEAEQLEPR                                    YVLEEAEQLEPR   

                                                Gene Names Organism_protein  \
Protein Accession Peptide                                                     
Q99453            AAAAAAAAAK                        PHOX2B     Homo sapiens   
P37108            AAAAAAAAAPAAAATAPTTAATTAATAAQ      SRP14     Homo sapiens   
                  FQMAYSNLLR                         SRP14     Homo sapiens   
                  GTVEGFEPADNK                       SRP14     Homo sapiens   
                  ISTVVSSK                           SRP14     Homo sapiens   
...                                                    ...              ...   
Q14865            YPESLSR                           ARID5B     Homo sapiens   
Q9BQP7            YSNLVQSVLSSR                       MGME1     Homo sapiens   
O95167            YSVMINK                           NDUFA3     Homo sapiens   
Q9UBM7            YTAAVPYR                           DHCR7     Homo sapiens   
Q8NAV1            YVLEEAEQLEPR                     PRPF38A     Homo sapiens   

                                                 Intensity_Ex_Auto_J3_30umTB_2ngQC_60m_1-calib_protein  \
Protein Accession Peptide                                                                                
Q99453            AAAAAAAAAK                                                               0.00000       
P37108            AAAAAAAAAPAAAATAPTTAATTAATAAQ                                        90183.00651       
                  FQMAYSNLLR                                                               0.00000       
                  GTVEGFEPADNK                                                             0.00000       
                  ISTVVSSK                                                            253830.39650       
...                                                                                            ...       
Q14865            YPESLSR                                                              57361.70404       
Q9BQP7            YSNLVQSVLSSR                                                             0.00000       
O95167            YSVMINK                                                                  0.00000       
Q9UBM7            YTAAVPYR                                                             32908.93552       
Q8NAV1            YVLEEAEQLEPR                                                             0.00000       

                                                 Intensity_Ex_Auto_J3_30umTB_2ngQC_60m_2-calib_protein  \
Protein Accession Peptide                                                                                
Q99453            AAAAAAAAAK                                                               0.00000       
P37108            AAAAAAAAAPAAAATAPTTAATTAATAAQ                                       173906.74480       
                  FQMAYSNLLR                                                               0.00000       
                  GTVEGFEPADNK                               

In [109]:
# let's test this with ms fragger
msfragger_protein_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\msfragger\\combined_protein.tsv"
msfragger_peptide_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\msfragger\\combined_peptide.tsv"
msfragger_outfile_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\msfragger\\peptideQ_and_protein.tsv"
try3 = controller(file_type='msfragger', peptideQ_file_path=msfragger_peptide_file_path, protein_file_path=msfragger_protein_file_path, peptideQ_and_protein_file_path=msfragger_outfile_path)
try3

Dataframe saved.


Prev AA  \
Protein Accession                                  Peptide                                 
Q86U42                                             AAAAAAAAAAGAAGGR                    M   
                                                   ELQNEVEK                            K   
                                                   GFAYIEFSDK                          K   
                                                   TSLALDESLFR                         R   
P37108                                             AAAAAAAAAPAAAATAPTTAATTAATAAQ       K   
...                                                                                  ...   
O43768                                             YFDSGDYNMAK                         K   
P61964                                             YILAATLDNTLK                        K   
P50402                                             YNIPHGPVVGSTR                       R   
contam_sp|P02788|TRFL_HUMAN Lactotransferrin OS... YYGYTGAFR                           R   
Q05639                                             YYITIIDAPGHR                        K   

                                                                                 Next AA  \
Protein Accession                                  Peptide                                 
Q86U42                                             AAAAAAAAAAGAAGGR                    G   
                                                   ELQNEVEK                            Q   
                                                   GFAYIEFSDK                          E   
                                                   TSLALDESLFR                         G   
P37108                                             AAAAAAAAAPAAAATAPTTAATTAATAAQ       -   
...                                                                                  ...   
O43768                                             YFDSGDYNMAK                         A   
P61964                                             YILAATLDNTLK                        L   
P50402                                             YNIPHGPVVGSTR                       R   
contam_sp|P02788|TRFL_HUMAN Lactotransferrin OS... YYGYTGAFR                           C   
Q05639                                             YYITIIDAPGHR                        D   

                                                                                  Start  \
Protein Accession                                  Peptide                                
Q86U42                                             AAAAAAAAAAGAAGGR                   2   
                                                   ELQNEVEK                         138   
                                                   GFAYIEFSDK                       214   
                                                   TSLALDESLFR                      228   
P37108                                             AAAAAAAAAPAAAATAPTTAATTAATAAQ    108   
...                                                                                 ...   
O43768                                             YFDSGDYNMAK                       64   
P61964                                             YILAATLDNTLK                     228   
P50402                                             YNIPHGPVVGSTR                     19   
contam_sp|P02788|TRFL_HUMAN Lactotransferrin OS... YYGYTGAFR                        544   
Q05639                                             YYITIIDAPGHR                      85   

                                                                                  End  \
Protein Accession                                  Peptide                              
Q86U42                                             AAAAAAAAAAGAAGGR                17   
                                                   ELQNEVEK                       145   
                                                   GFAYIEFSDK                     223   
                                                   TSLALDE